In [20]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch

# Load saved model & tokenizer
model_path = 'distilbert_airline'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.eval()

# Example text
text = "Can you show me available flights to Paris in December?"

# Tokenize
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Predict
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()

# Get label using id2label from model config
predicted_label = model.config.id2label[predicted_class_id]

print("Predicted intent:", predicted_label)


Predicted intent: Flights Info


In [36]:

import gradio as gr
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

# Load model and tokenizer
model_path = "distilbert_airline"  # your saved model folder
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.eval()

# Prediction function
def predict_intent(text):
    if not text.strip():
        return "Please enter some text."
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_id = torch.argmax(logits, dim=1).item()
    return model.config.id2label[predicted_id]

# Gradio interface
iface = gr.Interface(
    fn=predict_intent,
    inputs=gr.Textbox(lines=3, placeholder="Enter your query here..."),
    outputs="text",
    title="Airline Intent Classification"
)

# Launch app with public link
iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7868


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on public URL: https://a1afe4938436ea80b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import gradio as gr
import pandas as pd
import os

dataset_path = ".gradio/flagged/dataset3.csv"
feedback_path = ".gradio/flagged/feedback.csv"

# Load dataset3.csv and prepare for dashboard
def load_flagged():
    if os.path.exists(dataset_path):
        df = pd.read_csv(dataset_path, dtype=str)
    else:
        df = pd.DataFrame(columns=["text","output","timestamp"])
    # Add 'crted' column for admin input
    if "crted" not in df.columns:
        df["crted"] = ""
    # Ensure empty strings instead of NaN
    df = df.fillna("")
    return df[["text","output","crted","timestamp"]]

# Save only rows where crted is filled to feedback.csv
def save_feedback(df):
    # Select only rows where admin filled crted
    df_to_save = df[df["crted"].notna() & (df["crted"] != "")]
    if df_to_save.empty:
        return df
   
    # Prepare feedback format: text, correct_label, timestamp
    feedback_df = df_to_save[["text","crted","timestamp"]].rename(
        columns={"crted":"correct_label","timestamp":"timestamp"}
    )
    
    # Load existing feedback.csv if exists
    if os.path.exists(feedback_path):
        existing_feedback = pd.read_csv(feedback_path, dtype=str)
        feedback_df = pd.concat([existing_feedback, feedback_df], ignore_index=True)
        feedback_df = feedback_df.drop_duplicates(subset="text", keep="last")
    
    feedback_df.to_csv(feedback_path, index=False)
    
    # Remove saved rows from dataset3.csv
    dataset_df = pd.read_csv(dataset_path, dtype=str)
    dataset_df = dataset_df[~dataset_df["text"].isin(df_to_save["text"])]
    dataset_df.to_csv(dataset_path, index=False)
    
    # Reload updated table
    return load_flagged()
# Admin UI
with gr.Blocks() as demo:
    gr.Markdown("## Admin Panel: Correct Predicted Labels")
    
    table = gr.Dataframe(
        value=load_flagged(),
        headers=["text","output","crted","timestamp"],
        datatype=["str","str","str","str"],
        interactive=True,
        row_count=(1, "dynamic")  # allow deletion
    )

    save_btn = gr.Button("Save Corrections")
    save_btn.click(fn=save_feedback, inputs=[table], outputs=[table])

    refresh_btn = gr.Button("Refresh Table")
    refresh_btn.click(fn=load_flagged, inputs=[], outputs=[table])

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7865


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on public URL: https://a65ec8dd2d872148b3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
!pip uninstall -y tensorflow keras tf-keras

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
!pip install torch torchvision torchaudio transformers datasets


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
